# Wiscard Expenses

## 1) screen scrape
Start by refreshing the `wiscard.txt` file with a copy/paste of the data for the current month from the Wiscard site.

https://services.jsatech.com/index.php?cid=120

Copying the data from the "printed" verison of the data that appears in the pop-up window, and using

`pbpaste > wiscard.txt`

seems to have done the trick to preserve the tab delimiters.  Thus allowing awk to use \t as a field separator.  This might work with the data straight from the regular web page report, but I'll try that another time.

## 2) grep for expenses only

Filtering out payments, laundry, and any expenses from 'UBS', the University Book Store...

In [82]:
!egrep -v "Laundry|Payment|UBS" wiscard.txt > expenses.txt

## 3) load dataframe

In [83]:
import pandas as pd

expenses = pd.read_table('expenses.txt',
                         sep='\t',
                         header=None,
                         usecols=[0, 1, 2],
                         names=['datetime', 'location','charge'],
                         parse_dates=['datetime'],
                         date_parser=lambda x: pd.to_datetime('15/'+x, format="%y/%m/%d %H:%M %p")
                        )


expenses.head()

,datetime,location,charge
0,2015-08-31 03:23:00,Urban Slice 2 - Charge,-1.94
1,2015-09-01 05:29:00,Four Lakes Market 6 - Charge,-5.75
2,2015-09-02 08:23:00,Carson's Market 1 - Charge,-1.32
3,2015-09-02 10:33:00,Memorial Lib Ref - Charge,-1.40
4,2015-09-02 11:18:00,Harvest Grains 2 - Charge,-6.89


## 4) calulate spending by day and daily average

In [93]:
datetime_col = pd.DatetimeIndex(expenses.datetime)
spending_by_day = expenses.groupby([datetime_col.month,datetime_col.day]).charge.sum()
daily_average = spending_by_day.mean()
daily_average

-11.155967741935482